# Câu 1: Tokenizer theo BPE (Byte-Pair Encoding)

## 1. Cài đặt thư viện

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import re
from bs4 import BeautifulSoup
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec, KeyedVectors
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Cài đặt spaCy và tải mô hình ngôn ngữ tiếng Anh
nlp = spacy.load('en_core_web_sm')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 2. Tiền xử lý dữ liệu

In [ ]:
# Cài đặt thư viện Kaggle
!pip install kaggle
# Tải lên tệp kaggle.json của bạn chứa thông tin xác thực API
from google.colab import files
files.upload()
# Di chuyển tệp đã tải lên vào thư mục cần thiết
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
# Thiết lập quyền truy cập
!chmod 600 ~/.kaggle/kaggle.json
# Bây giờ bạn có thể tải tập dữ liệu bằng lệnh API Kaggle
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
# Giải nén tập dữ liệu
!unzip imdb-dataset-of-50k-movie-reviews.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 78% 20.0M/25.7M [00:00<00:00, 86.4MB/s]
100% 25.7M/25.7M [00:00<00:00, 84.1MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
# Tải tập dữ liệu từ Kaggle
df = pd.read_csv("/content/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
# In số lượng mẫu của từng nhãn
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [ ]:
# Lấy một phần của dữ liệu để xử lý
data = df.head(1000)
data["sentiment"].value_counts()

sentiment
positive    501
negative    499
Name: count, dtype: int64

In [ ]:
# Tải stopwords của NLTK
stopwords = set(stopwords.words('english'))
def preprocess_text(text):
    # Loại bỏ thẻ HTML
    #text = BeautifulSoup(text, "html.parser").get_text()
    # Loại bỏ ký tự đặc biệt và số
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Chuyển đổi thành chữ thường
    text = text.lower()
    # Tách từ
    tokens = word_tokenize(text)
    # Loại bỏ stopwords
    tokens = [word for word in tokens if word not in stopwords]
    # Kết hợp các từ lại thành một câu
    clean_text = ' '.join(tokens)
    return clean_text

# Áp dụng tiền xử lý vào cột 'review'
data.loc[:, 'review'] = data['review'].apply(preprocess_text)
data.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ho...,positive
1,wonderful little production br br filming tech...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake thinks zombie...,negative
4,petter mattei love time money visually stunnin...,positive


## 3. Xây dựng mô hình



### Mô hình 1: Không sử dụng BPE (Word2vec)

In [ ]:
# Vector semantics sử dụng Word2Vec
from gensim.models import Word2Vec, KeyedVectors
#'review' là cột chứa các đánh giá đã được xử lý trước
data.loc[:, 'tokenized'] = data['review'].apply(lambda x: word_tokenize(x))
# Loại bỏ các câu đã được mã hóa thành các từ riêng lẻ
tokenized_sentences = data['tokenized'][data['tokenized'].apply(len) > 0]
# Huấn luyện mô hình Word2Vec
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# Chuyển đổi đoạn văn thành vectors sử dụng Word2Vec (trung bình của các vectors từng từ)
def get_vector(sentence):
    vectors = [word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]
    if not vectors:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(vectors, axis=0)

# Giả sử tokenized_sentences là một cột trong DataFrame của bạn
data.loc[:, 'word2vec'] = data['tokenized'].apply(get_vector)

In [ ]:
data.head()

,review,sentiment,tokenized,word2vec
0,one reviewers mentioned watching oz episode ho...,positive,"[one, reviewers, mentioned, watching, oz, epis...","[-0.088796884, 0.23938274, 0.15620062, 0.11873..."
1,wonderful little production br br filming tech...,positive,"[wonderful, little, production, br, br, filmin...","[-0.11023495, 0.30169937, 0.19087982, 0.155272..."
2,thought wonderful way spend time hot summer we...,positive,"[thought, wonderful, way, spend, time, hot, su...","[-0.107855625, 0.2986823, 0.19135576, 0.150103..."
3,basically family little boy jake thinks zombie...,negative,"[basically, family, little, boy, jake, thinks,...","[-0.1258983, 0.34083784, 0.2140168, 0.17825179..."
4,petter mattei love time money visually stunnin...,positive,"[petter, mattei, love, time, money, visually, ...","[-0.1296892, 0.35634145, 0.2278476, 0.17872715..."


In [ ]:
# Chia dữ liệu thành các tập huấn luyện và kiểm tra
X = np.vstack(data['word2vec'].to_numpy())
y = data['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo bộ phân loại SVM
svm_classifier = SVC(C=1, kernel='rbf', probability=True)

# Huấn luyện mô hình với dữ liệu huấn luyện của bạn
svm_classifier.fit(X_train, y_train)

# Dự đoán sử dụng mô hình đã huấn luyện
predictions_word2vec = svm_classifier.predict(X_test)

In [ ]:
# Tính toán các chỉ số đánh giá
accuracy_word2vec = accuracy_score(y_test, predictions_word2vec)

print("Đánh giá mô hình Word2Vec")
print("Độ chính xác:", accuracy_word2vec)
print("Báo cáo phân loại:")
print(classification_report(y_test, predictions_word2vec, target_names=['negative', 'positive'], zero_division='warn'))

Đánh giá mô hình Word2Vec
Độ chính xác: 0.47
Báo cáo phân loại:
              precision    recall  f1-score   support

    negative       0.48      0.27      0.35       104
    positive       0.46      0.69      0.55        96

    accuracy                           0.47       200
   macro avg       0.47      0.48      0.45       200
weighted avg       0.47      0.47      0.45       200



### Mô hình 2: Sử dụng BPE

In [ ]:
# Cài đặt transformers
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.metrics import accuracy_score, classification_report

# Tải mô hình và tokenizer BERT đã được đào tạo trước
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize và tiền xử lý dữ liệu đầu vào, thêm hoặc cắt bớt để đạt đến max_length
def tokenize_data(text):
    return tokenizer.encode_plus(text,
                                  add_special_tokens=True,
                                  max_length=128,
                                  padding='max_length' if len(tokenizer.tokenize(text)) <= max_length else 'longest',
                                  truncation=True,
                                  return_attention_mask=True,
                                  return_tensors='pt')

# Tokenize và tiền xử lý dữ liệu đầu vào
tokenized_data = data['review'].apply(tokenize_data)

In [ ]:
# Chuyển đổi nhãn cảm xúc sang giá trị số
label_map = {'positive': 1, 'negative': 0}
y_numerical = data['sentiment'].map(label_map)

# Chuẩn bị tensors cho đầu vào của mô hình BERT
input_ids = torch.cat([tokenized_data[i]['input_ids'] for i in range(len(tokenized_data))], dim=0)
attention_masks = torch.cat([tokenized_data[i]['attention_mask'] for i in range(len(tokenized_data))], dim=0)
labels = torch.tensor(y_numerical.values)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
from sklearn.model_selection import train_test_split
input_ids_train, input_ids_test, attention_masks_train, attention_masks_test, labels_train, labels_test = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42
)

# Fine-tune mô hình BERT
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 32

train_data = TensorDataset(input_ids_train, attention_masks_train, labels_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(input_ids_test, attention_masks_test, labels_test)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
from transformers import get_linear_schedule_with_warmup
import torch.optim as optim

# Định nghĩa bộ tối ưu hóa và lên lịch biến đổi tốc độ học
optimizer = optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(train_dataloader) * 3)

# Vòng lặp huấn luyện
model.train()
for epoch in range(3):
    for batch in train_dataloader:
        input_ids_batch, attention_masks_batch, labels_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch, labels=labels_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

# Đánh giá
model.eval()
predictions_bert = []
true_labels = []

for batch in test_dataloader:
    input_ids_batch, attention_masks_batch, labels_batch = batch
    with torch.no_grad():
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).tolist()
    true_labels.extend(labels_batch.tolist())
    predictions_bert.extend(predictions)

In [ ]:
# Tính toán các chỉ số đánh giá
accuracy_bert = accuracy_score(true_labels, predictions_bert)

# In ra các chỉ số đánh giá
print("Đánh giá mô hình BERT:")
print("Độ chính xác:", accuracy_bert)
print("Báo cáo phân loại:")
print(classification_report(true_labels, predictions_bert, target_names=['negative', 'positive'], zero_division='warn'))

Đánh giá mô hình BERT:
Độ chính xác: 0.75
Báo cáo phân loại:
              precision    recall  f1-score   support

    negative       0.66      0.93      0.77        46
    positive       0.91      0.59      0.72        54

    accuracy                           0.75       100
   macro avg       0.79      0.76      0.75       100
weighted avg       0.80      0.75      0.74       100



## 4. Phân loại nhãn cho một review mới

In [ ]:
# Nhập câu từ bàn phím
input = 'A film very good'

**Với mô hình không sử dụng BPE**

In [ ]:
def preprocess_word2vec(sentence):
    tokenized = word_tokenize(sentence)
    vector = get_vector(tokenized)
    return vector.reshape(1, -1)

vector_word2vec = preprocess_word2vec(input)
predict_label_word2vec = svm_classifier.predict(vector_word2vec)
predict_proba_word2vec = svm_classifier.predict_proba(vector_word2vec)

print("Nhãn dự đoán từ Word2Vec:", predict_label_word2vec[0])
print("Phần trăm chính xác:", predict_proba_word2vec.max() * 100, "%")

Nhãn dự đoán từ Word2Vec: negative
Phần trăm chính xác: 51.34552881936426 %


**Với mô hình sử dụng BPE**

In [ ]:
def preprocess_bert(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", max_length=128, truncation=True, padding=True)
    return inputs

preprocessed_input = preprocess_bert(input)

with torch.no_grad():
    outputs = model(**preprocessed_input)
    logits = outputs.logits

predicted_label_bert = torch.argmax(logits, dim=1).item()

softmax = torch.nn.Softmax(dim=1)
probs = softmax(logits)

print("Nhãn dự đoán từ Bert:", "positive" if predicted_label_bert == 1 else "negative")
print("Phần trăm chính xác:", probs[0][predicted_label_bert].item() * 100, "%")

Nhãn dự đoán từ Bert: negative
Phần trăm chính xác: 68.03271770477295 %


# Câu 2:

## Chuẩn bị dữ liệu

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

# URL của tập tin văn bản
url = "https://raw.githubusercontent.com/binhvq/news-corpus/master/sample/demo-full.txt"

# Hàm để tải và xử lý tập tin văn bản
def fetch_and_split_text(url):
    # Tải nội dung của URL
    response = requests.get(url)
    if response.status_code != 200:
        return []

    # Nội dung của tập tin văn bản
    text_content = response.text

    # Tách văn bản thành các câu sử dụng biểu thức chính quy
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text_content)
    return sentences

# Tải và tách văn bản
data = fetch_and_split_text(url)
print(data[:10])  # Hiển thị 10 câu đầu tiên để kiểm tra
print(type(data))

['Chây ì nộp phạt nguội.', "Hàng chục ngàn phương tiện bị ghi hình vi phạm luật giao thông ở TP.HCM, bị 'bêu tên' nhưng chủ vẫn không chịu nộp phạt.", 'Trên cổng thông tin điện tử của Công an TP.HCM (CATP), mục thông tin về phương tiện vi phạm hành chính qua hình ảnh (từ ngày 4.1.2017 - 4.1.2018), có ghi nhận biển số xe, lỗi vi phạm, ngày vi phạm của 34.118 phương tiện (ô tô) chưa nộp phạt.', 'Đây là các phương tiện vi phạm được camera (di động hoặc cố định) của CATP ghi hình phạt nguội .', 'Điều đáng nói, dù Phòng CSGT đường bộ - đường sắt (PC67), CATP nhiều lần gửi giấy thông báo vi phạm về công an địa phương nhưng chủ hoặc người điều khiển phương tiện vẫn chưa thực hiện quyết định xử phạt hành chính.', 'Phổ biến nhất là lỗi đỗ không đúng nơi quy định.', 'Chẳng hạn từ tháng 1 - 7.2017, ô tô BS: 14A-130...', '23 lần đỗ không đúng nơi quy định trên đường Hàm Nghi, Q.1; từ tháng 3 - 10.2017, ô tô BS: 30S-087...', '34 lần đỗ trên đường Nguyễn Cư Trinh, Q.1; từ tháng 4 - 10.2017, ô tô BS:

Chuẩn bị danh sách các lỗi Tiếng Việt thường gặp

In [ ]:
vowel = list('aáàảãạăắằẳẵặâấầẩẫậeéèẻẽẹêếềểễệiíìỉĩịoóòỏõọôốồổỗộơớờởỡợuúùủũụưứừửữựyýỳỷỹỵAÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬEÉÈẺẼẸÊẾỀỂỄỆIÍÌỈĨỊOÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢUÚÙỦŨỤƯỨỪỬỮỰYÝỲỶỸỴ')
full_letters = vowel + list('bcdđfghjklmnpqrstvwxzBCDĐFGHJKLMNPQRSTVWZX')

# Những lỗi thường gặp trong tiếng Việt
  # Lỗi đánh máy
typo = {
    'á': 'as', 'à': 'af', 'ả': 'ar', 'ã': 'ax', 'ạ': 'aj', 'ắ': 'aws', 'ằ': 'awf', 'ẳ': 'awr', 'ẵ': 'awx', 'ặ': 'awj',
    'ấ': 'aas', 'ầ': 'aaf', 'ẩ': 'aar', 'ẫ': 'aax', 'ậ': 'aaj', 'é': 'es', 'è': 'ef', 'ẻ': 'er', 'ẽ': 'ex', 'ẹ': 'ej',
    'ế': 'ees', 'ề': 'eef', 'ể': 'eer', 'ễ': 'eex', 'ệ': 'eej', 'í': 'is', 'ì': 'if', 'ỉ': 'ir', 'ĩ': 'ix', 'ị': 'ij',
    'ó': 'os', 'ò': 'of', 'ỏ': 'or', 'õ': 'ox', 'ọ': 'oj', 'ố': 'oos', 'ồ': 'oof', 'ổ': 'oor', 'ỗ': 'oox', 'ộ': 'ooj',
    'ớ': 'ows', 'ờ': 'owf', 'ở': 'owr', 'ỡ': 'owx', 'ợ': 'owj', 'ú': 'us', 'ù': 'uf', 'ủ': 'ur', 'ũ': 'ux', 'ụ': 'uj',
    'ứ': 'uws', 'ừ': 'uwf', 'ử': 'uwr', 'ữ': 'uwx', 'ự': 'uwj', 'ý': 'ys', 'ỳ': 'yf', 'ỷ': 'yr', 'ỹ': 'yx', 'ỵ': 'yj',
    'Á': 'As', 'À': 'Af', 'Ả': 'Ar', 'Ã': 'Ax', 'Ạ': 'Aj', 'Ắ': 'Aws', 'Ằ': 'Awf', 'Ẳ': 'Awr', 'Ẵ': 'Awx', 'Ặ': 'Awj',
    'Ấ': 'Aas', 'Ầ': 'Aaf', 'Ẩ': 'Aar', 'Ẫ': 'Aax', 'Ậ': 'Aaj', 'É': 'Es', 'È': 'Ef', 'Ẻ': 'Er', 'Ẽ': 'Ex', 'Ẹ': 'Ej',
    'Ế': 'Ees', 'Ề': 'Eef', 'Ể': 'Eer', 'Ễ': 'Eex', 'Ệ': 'Eej', 'Í': 'Is', 'Ì': 'If', 'Ỉ': 'Ir', 'Ĩ': 'Ix', 'Ị': 'Ij',
    'Ó': 'Os', 'Ò': 'Of', 'Ỏ': 'Or', 'Õ': 'Ox', 'Ọ': 'Oj', 'Ố': 'Oos', 'Ồ': 'Oof', 'Ổ': 'Oor', 'Ỗ': 'Oox', 'Ộ': 'Ooj',
    'Ớ': 'Ows', 'Ờ': 'Owf', 'Ở': 'Owr', 'Ỡ': 'Owx', 'Ợ': 'Owj', 'Ú': 'Us', 'Ù': 'Uf', 'Ủ': 'Ur', 'Ũ': 'Ux', 'Ụ': 'Uj',
    'Ứ': 'Uws', 'Ừ': 'Uwf', 'Ử': 'Uwr', 'Ữ': 'Uwx', 'Ự': 'Uwj', 'Ý': 'Ys', 'Ỳ': 'Yf', 'Ỷ': 'Yr', 'Ỹ': 'Yx', 'Ỵ': 'Yj'
}

  # Lỗi vùng miền (phát âm sai)
region = {
    'ả': 'ã', 'ã': 'ả', 'ỏ': 'õ', 'õ': 'ỏ','ỉ': 'ĩ', 'ĩ': 'ỉ', 'ủ': 'ũ', 'ũ': 'ủ', 'ỷ': 'ỹ', 'ỹ': 'ỷ', 'Ả': 'Ã',
    'Ã': 'Ả', 'Ỏ': 'Õ', 'Õ': 'Ỏ', 'Ỉ': 'Ĩ', 'Ĩ': 'Ỉ', 'Ủ': 'Ũ', 'Ũ': 'Ủ','Ỷ': 'Ỹ', 'Ỹ': 'Ỷ'
}

region2 = {
    'ch': 'tr', 'tr': 'ch', 'x': 's', 's': 'x', 'd': 'gi', 'gi': 'd', 'r': 'd', 'd': 'r', 'l': 'n', 'n': 'l', 'Ch': 'Tr',
    'Tr': 'Ch', 'X': 'S', 'S': 'X', 'D': 'Gi', 'Gi': 'D', 'R': 'D', 'D': 'R', 'L': 'N', 'N': 'L'
}
  # Lỗi viết tắt
acronym = {
    'ck': 'chồng', 'ôg': 'ông', 'e': 'em', 'đc': 'được', 'ko': 'không', 'bn': 'bạn', 'mik': 'mình', 'a': 'anh', 'v': 'và',
    'đk': 'được', 'dc': 'được', 'j': 'gì', 'r': 'rồi', 'nt': 'nhắn tin','tv': 'ti vi', 'kq': 'kết quả', 'mn': 'mọi người',
    'bh': 'bây giờ', 'tn': 'thế nào', 'bc': 'báo cáo', 'bt': 'bình thường', 'cmt': 'comment', 'ns': 'nói', 'st': 'status',
    'qc': 'quảng cáo', 'kc': 'khác', 'ht': 'hiện tại', 'gd': 'gia đình', 'k': 'không', 'c': 'con', 'vđ': 'vấn đề', 'tp': 'thành phố',
    'nh': 'nhà', 'h': 'hôm', 'n': 'năm', 'vẫn': 'vẫn', 'thấy': 'thấy', 'bn': 'bao nhiêu', 'cv': 'công việc', 'thk': 'thích'
}

Hàm phát sinh lỗi chính tả thường gặp

In [ ]:
!pip install unidecode

In [ ]:
import re
import numpy as np
from unidecode import unidecode

# Hàm để tạo ra từ viết tắt trong câu
def _teen_code(sentence):
  random = np.random.uniform(0,1,1)[0]
  new_sentence = str(sentence)

  if random > 0.5:
    for word in acronym.keys():
      random2 = np.random.uniform(0,1,1)[0]
      if random2 < 0.5:
        new_sentence = re.sub(word, acronym[word], new_sentence)
    return new_sentence
  else:
    return sentence

# Hàm thêm các lỗi bằng xác suất ngẫu nhiên
def _add_noise(sentence):
    sentence = _teen_code(sentence)
    noisy_sentence = ''

    i = 0
    while i < len(sentence):
        if sentence[i] not in full_letters:
            noisy_sentence += sentence[i]
        else:
            random = np.random.uniform(0, 1, 1)[0]
            if random <= 0.94:
                noisy_sentence += sentence[i]
            elif random <= 0.985:
                if sentence[i] in typo.keys():
                    if sentence[i] in region.keys():
                        random2 = np.random.uniform(0, 1, 1)[0]
                        if random2 <= 0.4:
                            noisy_sentence += typo[sentence[i]]
                        elif random2 < 0.8:
                            noisy_sentence += region[sentence[i]]
                        elif random2 < 0.95:
                            noisy_sentence += unidecode(sentence[i])  # Sửa đổi tại đây
                        else:
                            noisy_sentence += sentence[i]
                    else:
                        random3 = np.random.uniform(0, 1, 1)[0]
                        if random3 <= 0.6:
                            noisy_sentence += typo[sentence[i]]
                        elif random3 < 0.9:
                            noisy_sentence += unidecode(sentence[i])  # Sửa đổi tại đây
                        else:
                            noisy_sentence += sentence[i]
                elif i == 0 or sentence[i-1] not in full_letters:
                    random4 = np.random.uniform(0, 1, 1)[0]
                    if random4 <= 0.9:
                        if i < len(sentence) - 1 and sentence[i] in region2.keys() and sentence[i+1] in vowel:
                            noisy_sentence += region2[sentence[i]]
                        elif i < len(sentence) - 2 and sentence[i:i+2] in region2.keys() and sentence[i+2] in vowel:
                            noisy_sentence += region2[sentence[i:i+2]]
                            i += 1
                        else:
                            noisy_sentence += sentence[i]
                    else:
                        noisy_sentence += sentence[i]
                else:
                    new_random = np.random.uniform(0, 1, 1)[0]
                    if new_random <= 0.33 and i != len(sentence) - 1:
                        noisy_sentence += sentence[i + 1]
                        noisy_sentence += sentence[i]
                        i += 1
                    else:
                        noisy_sentence += sentence[i]
        i += 1
    return noisy_sentence

In [ ]:
_add_noise('Tôi sẽ quay lại vào một ngày gần nhất')

'Tôi sẽ quay lại vào một nămàgy gầnăm nămhàất'

In [ ]:
# Các ký tự có thể xuất hiện trong encoder
alphabet = ['\x00',' '] + list('0123456789') + full_letters
print(len(alphabet))
print(alphabet)

198
['\x00', ' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'á', 'à', 'ả', 'ã', 'ạ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'e', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'i', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'o', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'u', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'y', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'A', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'E', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'I', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'O', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'U', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Y', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'b', 'c', 'd', 'đ', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'z', 'B', 'C', 'D', 'Đ', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Z', 'X']


Tách câu thành các cụm từ

In [ ]:
phrases = []

for text in data:
  # Thay thế hoặc xóa bỏ các ký tự thừa
  for c in set(text):
    if re.match('\w',c) and c not in alphabet:
      uc = unidecode(c)
      if re.match('\w',uc) and uc not in alphabet:
        text = re.sub(c, '',text)
      else:
        text = re.sub(c,uc,text)
  phrases += re.findall(r'\w[\w\s]+',text)

phrases = [p.strip() for p in phrases if len(p.split()) > 1]
print(len(phrases))
phrases[-10:]

3712


['2017 đã bỏ nội dung của điều 5 Thông tư 15',
 '2014 về việc',
 'người bán',
 'tặng xe phải thông báo bằng văn bản đến cơ quan đã cấp Giấy chứng nhận đăng ký xe đó để theo dõi',
 'và nội dung',
 'chủ xe không thông báo thì tiếp tục phải chịu trách nhiệm trước pháp luật về chiếc xe đó đến khi tổ chức',
 'cá nhân mua',
 'được điều chuyển',
 'tặng xe làm thủ tục đăng ký sang tên',
 'di chuyển xe']

Tách các cụm từ thành danh sách 5-grams với maxlen = 39

In [ ]:
from nltk import ngrams

NGRAM = 5
MAXLEN = 39

list_ngrams = []
for p in phrases:
  list_p = p.split()
  if (len(list_p) >= NGRAM):
    for ngr in ngrams(p.split(), NGRAM):
      if len(' '.join(ngr)) <= MAXLEN:
        list_ngrams.append(' '.join(ngr))
  elif len(' '.join(list_p)) <= MAXLEN:
    list_ngrams.append(' '.join(list_p))
list_ngrams = list((list_ngrams))
print(len(list_ngrams))
list_ngrams[:10]

21906


['Chây ì nộp phạt nguội',
 'Hàng chục ngàn phương tiện',
 'chục ngàn phương tiện bị',
 'ngàn phương tiện bị ghi',
 'phương tiện bị ghi hình',
 'tiện bị ghi hình vi',
 'bị ghi hình vi phạm',
 'ghi hình vi phạm luật',
 'hình vi phạm luật giao',
 'vi phạm luật giao thông']

Hàm encoder và decoder

In [ ]:
def _encoder_data(text):
  x = np.zeros((MAXLEN, len(alphabet)))
  for i, c in enumerate(text[:MAXLEN]):
    x[i, alphabet.index(c)] = 1
  if i < MAXLEN - 1:
    for j in range(i+1, MAXLEN):
      x[j, 0] = 1
  return x

def _decoder_data(x):
  x = x.argmax(axis = -1)
  return ''.join(alphabet[i] for i in x)

In [ ]:
print(_encoder_data('nhập môn xử lý ngôn ngữ tự nhiên').shape)
print(_decoder_data(_encoder_data('nhập môn xử lý ngôn ngữ tự nhiên')))

(39, 198)
nhập môn xử lý ngôn ngữ tự nhiên       


## Xây dựng mô hình

In [ ]:
# Kiến trúc mạng neural nhân tạo seq2seq
from keras.models import Sequential
from keras.layers import Activation, TimeDistributed, Dense, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
encoder = LSTM(256, input_shape=(MAXLEN, len(alphabet)), return_sequences=True)

In [ ]:
decoder = Bidirectional(LSTM(256, return_sequences=True, dropout=0.2))

In [ ]:
model = Sequential()

model.add(encoder)
model.add(decoder)
model.add(TimeDistributed(Dense(256)))
model.add(Activation('relu'))
model.add(TimeDistributed(Dense(len(alphabet))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 39, 256)           465920    
                                                                 
 bidirectional_2 (Bidirecti  (None, 39, 512)           1050624   
 onal)                                                           
                                                                 
 time_distributed_4 (TimeDi  (None, 39, 256)           131328    
 stributed)                                                      
                                                                 
 activation_4 (Activation)   (None, 39, 256)           0         
                                                                 
 time_distributed_5 (TimeDi  (None, 39, 198)           50886     
 stributed)                                                      
                                                      

Tách dữ liệu thành tập training và validation

In [ ]:
from sklearn.model_selection import train_test_split
train_data, valid_data = train_test_split(list_ngrams, test_size=0.2, random_state=42)

In [ ]:
print(len(train_data))

17524


In [ ]:
print(len(valid_data))

4382


In [ ]:
# Chia tách dữ liệu để tránh tràn RAM
BATCH_SIZE = 512
EPOCHS = 10

def _generate_data(data, batch_size):
    current_index = 0
    while True:
        x, y = [], []
        for i in range(batch_size):
            y.append(_encoder_data(data[current_index]))
            x.append(_encoder_data(_add_noise(data[current_index])))
            current_index += 1
            if current_index >= len(data):
                current_index = 0
        yield np.array(x), np.array(y)

In [ ]:
train_generator = _generate_data(data=train_data, batch_size=BATCH_SIZE)
validation_generator = _generate_data(data=valid_data, batch_size=BATCH_SIZE)

## Huấn luyện mô hình

In [ ]:
H = model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=len(train_data) // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=len(valid_data) // BATCH_SIZE
)

# Lưu mô hình
model.save('model_{0:.4f}.h5'.format(H.history['val_accuracy'][-1]))

Epoch 1/10
34/34 [==============================] - 190s 5s/step - loss: 2.6025 - accuracy: 0.5289 - val_loss: 1.9323 - val_accuracy: 0.5759
Epoch 2/10
34/34 [==============================] - 178s 5s/step - loss: 1.9034 - accuracy: 0.5777 - val_loss: 1.8809 - val_accuracy: 0.5806
Epoch 3/10
34/34 [==============================] - 176s 5s/step - loss: 1.8782 - accuracy: 0.5796 - val_loss: 1.8721 - val_accuracy: 0.5803
Epoch 4/10
34/34 [==============================] - 180s 5s/step - loss: 1.8666 - accuracy: 0.5806 - val_loss: 1.8576 - val_accuracy: 0.5818
Epoch 5/10
34/34 [==============================] - 174s 5s/step - loss: 1.8563 - accuracy: 0.5811 - val_loss: 1.8518 - val_accuracy: 0.5835
Epoch 6/10
34/34 [==============================] - 181s 5s/step - loss: 1.8183 - accuracy: 0.5853 - val_loss: 1.7652 - val_accuracy: 0.5948
Epoch 7/10
34/34 [==============================] - 182s 5s/step - loss: 1.7104 - accuracy: 0.6029 - val_loss: 1.6183 - val_accuracy: 0.6196
Epoch 8/10
34

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Ứng dụng mô hình

In [ ]:
from collections import Counter
from keras.models import load_model
import nltk
import numpy as np
import re

model = load_model("model_0.7524.h5")
NGRAM = 5
MAXLEN = 39

Các hàm xử lý Tiếng Việt

In [ ]:
# Tách câu thành các ngram
def _nltk_ngrams(sentence, n, maxlen):
  list_ngrams = []
  list_words = sentence.split()
  num_words = len(list_words)

  if (num_words >= n):
    for ngram in nltk.ngrams(list_words, n):
      if len(' '.join(ngram)) <= maxlen:
        list_ngrams.append(ngram)
  else:
    list_ngrams.append(tuple(list_words))

  return list_ngrams

In [ ]:
_nltk_ngrams('Xuwr ly ngonn ngu tuwj nhiên', NGRAM, MAXLEN)

[('Xuwr', 'ly', 'ngonn', 'ngu', 'tuwj'),
 ('ly', 'ngonn', 'ngu', 'tuwj', 'nhiên')]

In [ ]:
# Dự đoán ngram bằng mô hình
def _guess(ngram):
  text = ' '.join(ngram)
  preds = model.predict(np.array([_encoder_data(text)]))

  return _decoder_data(preds[0]).strip('\x00')

In [ ]:
_guess(('Xuwr', 'lys', 'ngono', 'nguwx', 'tu'))

1/1 [==============================] - 0s 43ms/step


'uuu  lền nggn      '

In [ ]:
# Thêm dấu câu sau khi sửa lỗi chính tả
def _add_punctuation(text, corrected_text):
  list_punctuation = {}

  for (i, word) in enumerate(text.split()):
    if word[0] not in alphabet or word[-1] not in alphabet:
      start_punc = ''
      for c in word:
        if c in alphabet:
          break
        start_punc += c

      end_punc = ''
      for c in word[::-1]:
        if c in alphabet:
          break
        end_punc += c
      end_punc = end_punc[::-1]

      list_punctuation[i] = [start_punc, end_punc]
  result = ''
  for (i, word) in enumerate(corrected_text.split()):
    if i in list_punctuation:
      result += (list_punctuation[i][0]+word+list_punctuation[0][i])+' '
    else:
      result += word + ' '
  return result.strip()

In [ ]:
# Sửa lỗi chính tả trong câu
def _correct(text):
  new_text = re.sub(r'[^' + ''.join(alphabet) + ']', '', text)

  ngrams = list(_nltk_ngrams(new_text, NGRAM, MAXLEN))
  guessed_ngrams = list(_guess(ngram) for ngram in ngrams)
  candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM -1)]

  for nid, ngram in (enumerate(guessed_ngrams)):
    for wid, word in (enumerate(re.split('\s', ngram))):
      candidates[nid + wid].update([word])

  corrected_text = ' '.join(c.most_common(1)[0][0] for c in candidates if c)
  return _add_punctuation(text, corrected_text)

In [ ]:
text = 'Xuw ly ngon ngu tuwj nhiên'
_correct(text)

1/1 [==============================] - 0s 199ms/step


'uuu ly nghn ngu tế nnnn'

In [ ]:
text = input()
result = _correct(text)
print(result)

vk ck con cái
1/1 [==============================] - 0s 63ms/step
vi cá con cái
